In [ ]:
import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from googleapiclient.http import MediaFileUpload
import io
from googleapiclient.errors import HttpError
from dotenv import load_dotenv
import os

In [2]:
# Load environment variables from .env file
load_dotenv()

key_file_name = os.getenv('KEY_FILE_NAME')
folder_id = os.getenv('FOLDER_ID')
my_folder_id = os.getenv('MY_FOLDER_ID')

In [3]:
# It controls the access of the application. Currently it allows read and write both
scope = ['https://www.googleapis.com/auth/drive']

current_directory = os.getcwd()
service_account_json_key = os.path.join(current_directory, key_file_name)

credentials = service_account.Credentials.from_service_account_file(
                              filename=service_account_json_key, 
                              scopes=scope)

service = build('drive', 'v3', credentials=credentials)

In [4]:
# Takes file_id as input and puts it in example.jpg in the current directory
def download_image(file_id):
    request = service.files().get_media(fileId = file_id)
    download_file_path = 'temp.jpg'
    fh = io.FileIO(download_file_path, mode='wb')
    downloader = MediaIoBaseDownload(fh, request)
    
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        print(f"Download {int(status.progress() * 100)}%.")

In [5]:
# It takes the name of file to be uploaded on drive and path of image on local computer
def upload_image(file_name_on_drive, download_image_path):
    file_metadata = {'name': file_name_on_drive}  
    media = MediaFileUpload(download_image_path, mimetype='image/jpeg')
    file_metadata['parents'] = [my_folder_id]
    file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    print('File ID:', file.get('id'))

In [6]:
# Takes the path of file and deletes the image
def delete_image(file_path):
    try:
        os.remove(file_path)
        print(f"Filze '{file_path deleted successfully.")
    except FileNotFoundError:
        print(f"File '{file_path}' not found.")
    except PermissionError:
        print(f"Permission denied to delete file '{file_path}'.")
    except Exception as e:
        print(f"An error occurred: {e}")


In [7]:
# Reture the numerical part of image name
def convert_to_number(string):
    for i, char in enumerate(string):
        if not char.isdigit():
            return int(string[:i])
    return int(string)

In [ ]:
page_token = None
# Google drive can serve upto 1000 files in one request, page solves this issue by accessing the next in the order
while True:
    results = service.files().list(
        pageSize=1000,  
        fields="nextPageToken, files(id, name)",
        q=f"'{folder_id}' in parents",
        pageToken=page_token
    ).execute()

    files = results.get('files', [])
    
    if not files:
        print(f'No more files found in folder with ID: {folder_id}')
        break

    download_image_path = image_path = os.path.join(current_directory, 'temp.jpg') 
    apple_counter_map = {}

    mapping = {i: chr(i + ord('a')) for i in range(26)}
    
    # Print the name of files in the folder
    for file in files:
        image_id = file['id']
        image_name = file['name']
        apple_no = convert_to_number(image_name)
        if apple_counter_map.get(apple_no) == None:
            apple_counter_map[apple_no] = 0
        else:
            apple_counter_map[apple_no] += 1
        download_image(image_id)
        file_name_on_drive = str(apple_no) + mapping[apple_counter_map[apple_no]] +'.jpg'
        upload_image(file_name_on_drive, download_image_path)
        delete_image(download_image_path)
        

    # Get the next page token
    page_token = results.get('nextPageToken')

    if not page_token:
        break